<a href="https://colab.research.google.com/github/donghoney/Algorithm/blob/master/Assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.pyplot as plt
import numpy as np

file_data   = "mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)
list_vectors = np.empty(size_col * size_row, dtype=float)
list_test = np.empty(size_col * size_row, dtype=float)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    if count == 0:
      list_vectors = im_vector
    elif count < 6000:
      list_vectors = np.vstack([list_vectors, im_vector])
    elif count == 6000:
      list_test = im_vector
    else:
      list_test = np.vstack([list_test, im_vector])

    count += 1

u = np.random.normal(0, 1 , 785 * 196).reshape((785, 196))
v = np.random.normal(0, 1 , 197 * 49).reshape((197, 49))
w = np.random.normal(0, 1 , 50 * 10).reshape((50, 10))
bias = np.ones([6000, 1])
m = 6000
bias_test = np.ones([4000,1])
m_test = 4000
rate = 0.3

label_enc = np.array([[1,0,0,0,0,0,0,0,0,0]
             ,[0,1,0,0,0,0,0,0,0,0]
             ,[0,0,1,0,0,0,0,0,0,0]
             ,[0,0,0,1,0,0,0,0,0,0]
             ,[0,0,0,0,1,0,0,0,0,0]
             ,[0,0,0,0,0,1,0,0,0,0]
             ,[0,0,0,0,0,0,1,0,0,0]
             ,[0,0,0,0,0,0,0,1,0,0]
             ,[0,0,0,0,0,0,0,0,1,0]
             ,[0,0,0,0,0,0,0,0,0,1]])

cnt = []
Jlist = []
J_test_list = []
acc_test_list = []
acc_list = []

for i in range(1000):
  J = 0
  J_test = 0
  acc = 0
  acc_test = 0

  x_nobias = list_vectors
  x_test_nobias = list_test
  x = np.hstack([bias, list_vectors])
  x_test = np.hstack([bias_test, list_test])

  y = x.dot(u)
  y_sigma = 1 / (1 + np.exp(-1 * y))
  y_sigma_bias = np.hstack([bias, y_sigma])

  y_test = x_test.dot(u)
  y_sigma_test = 1 / (1 + np.exp(-1 * y_test))
  y_sigma_test_bias = np.hstack([bias_test, y_sigma_test])
  
  z = y_sigma_bias.dot(v)
  z_sigma = 1 / (1 + np.exp(-1 * z))
  z_sigma_bias = np.hstack([bias, z_sigma])

  z_test = y_sigma_test_bias.dot(v)
  z_sigma_test = 1 / (1 + np.exp(-1 * z_test))
  z_sigma_test_bias = np.hstack([bias_test, z_sigma_test])

  h_test = z_sigma_test_bias.dot(w)
  h_sigma_test = 1 / (1 + np.exp(-1 * h_test))

  h = z_sigma_bias.dot(w)
  h_sigma = 1 / (1 + np.exp(-1 * h))

  u_prime = np.zeros((785,196))
  v_prime = np.zeros((197,49))
  w_prime = np.zeros((50, 10))
  
  for j in range(6000):
    a = np.where(h_sigma[j] == max(h_sigma[j]))
    if list_label[j] == a[0][0]:
      acc += 1

  for j in range(6000):
    J += np.sum(-1 * label_enc[list_label[j]] * np.log(h_sigma[j]) - (1 - label_enc[list_label[j]]) * np.log(1 - h_sigma[j])) 

  Jlist.append(J / m)
  acc_list.append(acc / m)

  for j in range(4000):
    a = np.where(h_sigma_test[j] == max(h_sigma_test[j]))
    if list_label[j+6000] == a[0][0]:
      acc_test += 1

  for j in range(4000):
    J_test += np.sum(-1 * label_enc[list_label[j+6000]] * np.log(h_sigma_test[j]) - (1 - label_enc[list_label[j+6000]]) * np.log(1 - h_sigma_test[j])) 

  J_test_list.append(J_test / m_test)
  acc_test_list.append(acc_test / m_test)

  for j in range(6000):
    delta4 = h_sigma[j] - label_enc[list_label[j]] # 1x10
    delta4 = np.array([delta4])

    z_cal = z_sigma[j] * (1 - z_sigma[i])
    delta3 = delta4.dot(np.transpose(np.delete(w, (0), axis=0))) * np.array([z_cal])

    y_cal = y_sigma[i] * (1 - y_sigma[j])
    delta2 = delta3.dot(np.transpose(np.delete(v, (0), axis=0))) * np.array([y_cal])

    u_prime = u_prime + np.transpose(np.array([x[j]])).dot(delta2)
    v_prime = v_prime + np.transpose(np.array([y_sigma_bias[j]])).dot(delta3)
    w_prime = w_prime + np.transpose(np.array([z_sigma_bias[j]])).dot(delta4)

  u_prime = u_prime / 6000
  v_prime = v_prime / 6000
  w_prime = w_prime / 6000

  u = u - rate * u_prime
  v = v - rate * v_prime
  w = w - rate * w_prime
  
  cnt.append(i)


#
# plot first 150 images out of 10,000 with their labels
#
'''
f1 = plt.figure(1)

for i in range(150):

    label       = list_label[i]
    im_vector   = list_image[:, i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(10, 15, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)


#plt.show()

#
# plot the average image of all the images for each digit
#
f2 = plt.figure(2)

im_average  = np.zeros((size_row * size_col, 10), dtype=float)
im_count    = np.zeros(10, dtype=int)

for i in range(num_image):

    im_average[:, list_label[i]] += list_image[:, i]
    im_count[list_label[i]] += 1

for i in range(10):

    im_average[:, i] /= im_count[i]

    plt.subplot(2, 5, i+1)
    plt.title(i)
    plt.imshow(im_average[:,i].reshape((size_row, size_col)), cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

plt.show()
'''

In [0]:
x_test_nobias = list_test
x_test = np.hstack([bias_test, list_test])

y_test = x_test.dot(u)
y_sigma_test = 1 / (1 + np.exp(-1 * y_test))
y_sigma_test_bias = np.hstack([bias_test, y_sigma_test]) 

z_test = y_sigma_test_bias.dot(v)
z_sigma_test = 1 / (1 + np.exp(-1 * z_test))
z_sigma_test_bias = np.hstack([bias_test, z_sigma_test])

h_test = z_sigma_test_bias.dot(w)
h_sigma_test = 1 / (1 + np.exp(-1 * h_test))

correct_img = []
correct_label = []
incorrect_img = []
incorrect_label = []

correct_cnt = 0
incorrect_cnt = 0

for i in range(4000):
  a = np.where(h_sigma_test[i] == max(h_sigma_test[i]))
  if list_label[i+6000] == a[0][0]:
    if correct_cnt < 10:
      correct_cnt += 1
      correct_img.append(list_test[i])
      correct_label.append(a[0][0])
  else:
    if incorrect_cnt < 10:
      incorrect_cnt += 1
      incorrect_img.append(list_test[i])
      incorrect_label.append(a[0][0])

1. Plot the loss curve

In [0]:
plt.plot(cnt, Jlist, 'b')
plt.plot(cnt, J_test_list, 'r')

plt.show()

2. Plot the accuracy curve

In [0]:
plt.plot(cnt, acc_list, 'b')
plt.plot(cnt, acc_test_list, 'r')

plt.show()

3. Plot the accuracy value

In [0]:
print('training accuracy : {} %'.format(acc/m*100))
print('testing accuracy : {} %'.format(acc_test/m_test*100))

4. Plot the classification example

In [0]:
for i in range(10):

    label       = correct_label[i]
    im_vector   = correct_img[i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(2, 5, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)


plt.show()

In [0]:
for i in range(10):

    label       = incorrect_label[i]
    im_vector   = incorrect_img[i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(2, 5, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)


plt.show()